In [92]:
import datetime as dt
import numpy as np
import pandas as pd
import san
from san import Batch

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [113]:
projects = ['bitcoin', 'ethereum', 'santiment', 'ripple', 'bitcoin-cash', 'eos', 'stellar', 'litecoin', 
            'status', 'poet', 'cardano', 'waves', 'aeternity', 'golem-network-tokens', 'rchain']
from_date = '2018-01-01'
to_date = '2018-10-16'
interval='1d'

In [114]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = True
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [115]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

In [116]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').values,
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [117]:
results = []
for prj in projects:
    print(prj)
    res = san.get("prices/{}".format(prj), from_date=from_date, to_date=to_date, interval=interval)
    results.append(res[['priceUsd']])

# print(results)
combined_df = merge_dfs_on_column(results, projects, 'priceUsd')
combined_df.head(5)

bitcoin
ethereum
santiment
ripple
bitcoin-cash
eos
stellar
litecoin
status
poet
cardano
lykke
waves
aeternity
golem-network-tokens
rchain


,bitcoin,ethereum,santiment,ripple,bitcoin-cash,eos,stellar,litecoin,status,poet,cardano,lykke,waves,aeternity,golem-network-tokens,rchain
datetime,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,13673.978472,763.999472,5.470535,2.243406,2471.258785,8.855674,0.442389,229.334861,0.262006,0.091682,0.702413,0.224699,12.337522,1.297270,0.867287,0.922141
2018-01-02 00:00:00+00:00,14059.336111,874.942208,6.088624,2.354528,2596.368125,9.324071,0.529914,250.800965,0.286158,0.097040,0.752802,0.224538,12.987174,1.373983,1.044404,1.990383
2018-01-03 00:00:00+00:00,15176.018403,902.362417,6.357606,2.815152,2740.845104,9.162836,0.754399,251.501705,0.518981,0.098771,0.976108,0.215208,13.066307,1.437316,1.092280,2.154295
2018-01-04 00:00:00+00:00,15123.029167,981.240236,6.504272,3.536460,2452.888958,11.326202,0.843058,236.009563,0.594945,0.151228,1.210326,0.211396,12.489718,1.667780,1.113063,2.028169
2018-01-05 00:00:00+00:00,16289.108333,1017.884931,7.553974,3.170925,2480.441840,10.799145,0.706517,248.489177,0.532665,0.200966,1.051942,0.208966,12.500094,2.133962,1.022979,2.553073


In [118]:
df_scatter(combined_df, 'Cryptocurrency Prices (USD)', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale='log')

In [119]:
combined_df_2018 = combined_df[combined_df.index.year == 2018]
combined_df_2018.head(5)

,bitcoin,ethereum,santiment,ripple,bitcoin-cash,eos,stellar,litecoin,status,poet,cardano,lykke,waves,aeternity,golem-network-tokens,rchain
datetime,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,13673.978472,763.999472,5.470535,2.243406,2471.258785,8.855674,0.442389,229.334861,0.262006,0.091682,0.702413,0.224699,12.337522,1.297270,0.867287,0.922141
2018-01-02 00:00:00+00:00,14059.336111,874.942208,6.088624,2.354528,2596.368125,9.324071,0.529914,250.800965,0.286158,0.097040,0.752802,0.224538,12.987174,1.373983,1.044404,1.990383
2018-01-03 00:00:00+00:00,15176.018403,902.362417,6.357606,2.815152,2740.845104,9.162836,0.754399,251.501705,0.518981,0.098771,0.976108,0.215208,13.066307,1.437316,1.092280,2.154295
2018-01-04 00:00:00+00:00,15123.029167,981.240236,6.504272,3.536460,2452.888958,11.326202,0.843058,236.009563,0.594945,0.151228,1.210326,0.211396,12.489718,1.667780,1.113063,2.028169
2018-01-05 00:00:00+00:00,16289.108333,1017.884931,7.553974,3.170925,2480.441840,10.799145,0.706517,248.489177,0.532665,0.200966,1.051942,0.208966,12.500094,2.133962,1.022979,2.553073


In [120]:
combined_df_2018.pct_change().corr(method='pearson')

,bitcoin,ethereum,santiment,ripple,bitcoin-cash,eos,stellar,litecoin,status,poet,cardano,lykke,waves,aeternity,golem-network-tokens,rchain
bitcoin,1.000000,0.798296,0.828072,0.679324,0.791350,0.687739,0.718089,0.833384,0.650795,0.684654,0.762219,0.253824,0.772052,0.675340,0.723011,0.520129
ethereum,0.798296,1.000000,0.795204,0.687052,0.790840,0.706506,0.673120,0.786746,0.609241,0.650846,0.756036,0.305548,0.734805,0.716292,0.745035,0.645557
santiment,0.828072,0.795204,1.000000,0.670348,0.757452,0.645685,0.648808,0.786706,0.580345,0.693917,0.703501,0.220718,0.763896,0.643743,0.716783,0.553064
ripple,0.679324,0.687052,0.670348,1.000000,0.645303,0.649692,0.771991,0.646074,0.702280,0.630325,0.799887,0.196643,0.565606,0.528889,0.590680,0.417641
bitcoin-cash,0.791350,0.790840,0.757452,0.645303,1.000000,0.675054,0.628324,0.792241,0.564985,0.542737,0.674387,0.252306,0.711792,0.635793,0.661172,0.484112
eos,0.687739,0.706506,0.645685,0.649692,0.675054,1.000000,0.632162,0.644282,0.570643,0.601824,0.755835,0.250969,0.614751,0.710115,0.554452,0.406455
stellar,0.718089,0.673120,0.648808,0.771991,0.628324,0.632162,1.000000,0.644166,0.827090,0.572406,0.868714,0.210516,0.660433,0.557615,0.620103,0.491195
litecoin,0.833384,0.786746,0.786706,0.646074,0.792241,0.644282,0.644166,1.000000,0.577768,0.611542,0.696512,0.246524,0.725510,0.618436,0.663514,0.557122
status,0.650795,0.609241,0.580345,0.702280,0.564985,0.570643,0.827090,0.577768,1.000000,0.523870,0.843854,0.176721,0.565600,0.525343,0.570391,0.398474
poet,0.684654,0.650846,0.693917,0.630325,0.542737,0.601824,0.572406,0.611542,0.523870,1.000000,0.653997,0.137656,0.594277,0.601574,0.585357,0.435001


In [121]:
correlation_heatmap(combined_df_2018.pct_change(), "Cryptocurrency Correlations in 2018")